In [ ]:
df_mcl = pd.read_csv('/nfs/masi/gaor2/data/MCL/clinical_mcl/MCL20200605_withconfid.csv')
df_spore = pd.read_csv('/nfs/masi/gaor2/data/SPORE/Clinical_forspore/spore_withconfid.csv')
df_nlst = pd.read_csv('/nfs/masi/gaor2/data/NLST/Clinical_forspore/KaggleTrained/withlabel_withconfid.csv')


cancer = df_mcl['cancer'].tolist() + df_spore['cancer'].tolist() + df_nlst['cancer'].tolist()
ID  =df_mcl['id'].tolist() + df_spore['id'].tolist() + df_nlst['id'].tolist()
max_confid_score = df_mcl['max_confid_score'].tolist() + df_spore['max_confid_score'].tolist() + df_nlst['max_confid_score'].tolist()
source = ['mcl'] * len(df_mcl) + ['spore'] * len(df_spore) + ['nlst'] * len(df_nlst)
 
data = pd.DataFrame()

data['cancer'] = cancer
data['ID'] = ID
data['max_confid_score'] = max_confid_score
data['source'] = source

data.to_csv('/nfs/masi/gaor2/data/Cotrain/clinical_spore/threeset20200612.csv', index = False)


In [ ]:
# combine the three set for multipath training

df = pd.read_csv('/nfs/masi/gaor2/data/Cotrain/clinical_spore/mcl/ThreeSet_norm.csv')
df1 = pd.read_csv('/nfs/masi/gaor2/data/Cotrain/clinical_spore/threeset20200612.csv')
data = pd.DataFrame()
for i, item in df.iterrows():
    for factor in df.keys():
        if factor not in ['withimg0601', 'complete_factor', 'kaggle', 'sess', 'source', 'max_confid_score']:
            data[factor] =  df[factor].tolist() + [''] * len(df1)
        else:
            data['withimg0601'] = df['withimg0601'].tolist() + [1] * len(df1)
            data['complete_factor'] = df['complete_factor'].tolist() + [0] * len(df1)
            data['kaggle'] = df['kaggle'].tolist() +  df1['cancer'].tolist()
            data['sess'] = df['sess'].tolist() + df1['ID'].tolist()
            data['source'] = df['source'].tolist() + df1['source'].tolist()
            data['max_confid_score'] = [''] * len(df) + df1['max_confid_score'].tolist()
data.to_csv('/nfs/masi/gaor2/data/Cotrain/clinical_spore/mcl/ThreeSet_norm0612.csv', index = False)             

In [ ]:
# remove the duplicate sess 
df = pd.read_csv('/nfs/masi/gaor2/data/Cotrain/clinical_spore/mcl/ThreeSet_norm0612.csv')
exist = []
exist_sess = []
for i, item in df.iterrows(): 
    if item['sess'] != item['sess']:
        exist.append(0)
        continue
    if item['sess'] not in exist_sess:
        exist_sess.append(item['sess'])
        exist.append(0)
    else:
        exist.append(1)
df['exist_before'] = exist
df.to_csv('/nfs/masi/gaor2/data/Cotrain/clinical_spore/mcl/ThreeSet_norm0612.csv', index = False)  

In [ ]:
pbb_root= '/nfs/masi/NLST/DSB_File/diag/bbox'
df = pd.read_csv('/nfs/masi/gaor2/data/NLST/Clinical_forspore/KaggleTrained/withlabel.csv')
max_confid_score = []
for i, item in df.iterrows():
    #print (i, item['id'])
    pbb = np.load(pbb_root + '/' + item['id'] + '_pbb.npy')
    if pbb.shape[0] < 1:
        max_confid_score.append(-2)
        continue
    output = pbb[np.argsort(-pbb[:, 0])]
    max_confid_score.append(output[0][0])
df['max_confid_score'] = max_confid_score
df.to_csv('/nfs/masi/gaor2/data/NLST/Clinical_forspore/KaggleTrained/withlabel_withconfid.csv')

